In [2]:
import torch
from transformers import BertTokenizer
from util.util import *
from tqdm import tqdm
import seaborn as sns

In [3]:
filename = "/data/medg/misc/phuongpm/"

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
# traindata = JsonDataset(filename + "train1.0.json")

In [6]:
# devdata = JsonDataset(filename + "dev1.0.json")

In [7]:
# testdata = JsonDataset(filename + 'test1.0.json')

In [8]:
# lenps = []
# lenqs = []

# for datum in tqdm(traindata.dataset[DATA_KEY]):
#     doc = remove_entity_marks(datum[DOC_KEY][TITLE_KEY] + " " + datum[DOC_KEY][CONTEXT_KEY]).replace(
#                         "\n", " ").lower()
#     lenps.append(len(tokenizer.tokenize(doc)))
#     for qa in datum[DOC_KEY][QAS_KEY]:
#         q = remove_entity_marks(qa[QUERY_KEY]).replace("\n", " ").lower()
#         lenqs.append(len(tokenizer.tokenize(q)))

In [9]:
# sns.distplot(lenps)

In [10]:
# sampledata = JsonDataset(filename + 'sample1.0.json')

### Convert to SQuad

In [11]:
sampledata = load_json(filename + 'sample1.0.json')

In [13]:
sampledata.keys()

dict_keys(['version', 'data'])

In [14]:
sampledata[DATA_KEY][0]

{'document': {'context': 'Summary\n\nThis report describes a term newborn with BEG__isolated distortion in the left parietal bone__END without any BEG__other visible congenital anomaly__END , due to BEG__amniotic band disruption__END .\nA BEG__skull x-ray__END , BEG__ultrasound scan__END and BEG__subsequent MRI scan of the brain__END did not show any BEG__apparent distortion__END apart from BEG__depression__END and BEG__concavity in the left parietal bone__END .\nThe purpose of this case report is to raise awareness of this possible , mild outcome of this BEG__little - known entity__END , which may BEG__mimic caput succedaneum__END ( moulding of the presenting part in the birth canal during natural delivery ) , and to provide a historical and embryological background .\n\nBackground\n\nBEG__Amniotic band disruption syndrome__END is a rare entity which occurs in 1 in 1200 to 1 in 15 000 live births .\n1 It may cause a BEG__myriad of deformities of fetal body parts__END from BEG__mild de

In [56]:
data = []
for datum in tqdm(sampledata[DATA_KEY]):
    title = remove_entity_marks(datum[DOC_KEY][TITLE_KEY]).replace("\n", " ").lower()
    context = remove_entity_marks(datum[DOC_KEY][CONTEXT_KEY]).replace("\n", " ").lower()
    qas=[]
    for qa in datum[DOC_KEY][QAS_KEY]:
        a = ""
        answers = []
        for ans in qa[ANS_KEY]:
            if ans[ORIG_KEY] == "dataset":
                a = ans[TXT_KEY].lower()
        if ' '+ a not in context:
            continue
            
        start = 0
        while start < len(context) and start > -1:
            try:
                answer_start = context.index(' '+a, start+1)
            except:
                break
            answers.append({'answer_start': answer_start, 'text': a})
            start = answer_start
        qas.append({
            'answers':answers,
            'question':remove_entity_marks(qa[QUERY_KEY]).replace("\n", " ").lower().replace('@placeholder', 'what').replace(
    '.', '?').replace("▶ ",""),
            'id':qa[ID_KEY]
        })
    if len(qas) > 0:
        data.append({
            'title':title,
            'paragraphs':[{'context': context, 'qas':qas}]
        })  





100%|██████████| 20/20 [00:00<00:00, 3766.61it/s]A


In [57]:
save = {'data':data, 'version':sampledata[VERSION_KEY]}

In [58]:
save

{'data': [{'title': 'pulmonary alveolar microlithiasis',
   'paragraphs': [{'context': 'a 40 - year - old man presented with shortness of breath and cough . chest radiography showed extensive bilateral nodular opacities ( sandstorm - like ) predominantly in the middle and lower lobe ( fig 1a ) in the absence of metabolic abnormalities . pulmonary function test revealed restrictive pattern of ventilation . high resolution computed tomography ( ct ) scan showed ground glass and increased parenchymal attenuation with coarsened , coalescing , nodular alveolar infiltrates associated with thickening and nodularity of peribronchovascular , interlobular and subpleural interstitium ( fig 1b ) . the changes were more pronounced and dense in the subpleural centrilobular area . patent bronchi and interspersed subpleural emphysematous changes ( producing black pleural line ) were present . this was further confirmed by histopathological examination of the lung biopsy specimen which showed normal re

In [59]:
save_json(save,'/data/medg/misc/phuongpm/likesquad.json')